In [1]:
import time
from os import path
from pathlib import Path
from datetime import datetime

import astropy.units as u
import numpy as np
from astropy.table import Table, vstack, unique


from astroquery.vizier import Vizier
#from astroquery.ned.core import RemoteServiceError, TableParseError

import requests
from requests import Request, Session

from numpy import unique as uniq

from astropy.coordinates import SkyCoord
from astropy.table import Table

In [5]:
path_raw='../Datos/Raw/'
path_phot_ned = '../Datos/Phot_Tables_NED/'
path_phot_cds = '../Datos/Phot_Tables_CDS/' #In this path wi will save the photometry tables for each object
path_cds_vot = '../Datos/Phot_Tables_CDS/VOT/' 
Sample = Table.read(path_raw+'Obj_Ne-IR_Apr_06_2022.vot',format='votable')
Sample

Main_Id,Id_used_NED,RA,DEC,otype_txt,redshift,rvz_type,otypes,cds_url,SMB_sample_index
object,object,float64,float64,object,float64,str1,object,str93,float64
3C 234.0,3C 234.0,150.45634473807,28.7858176354,Sy1,0.18494,z,AGN|G |IR |QSO|Rad|Sy1|X |blu|gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=150.45634473807%2028.7858176354&-c.rs=5,283.0
M 84,M 84,186.2655972083333,12.886983138888887,Sy2,0.00327,z,* |AGN|G |GiC|GiG|IR |Rad|Sy2|UV |X |smm,http://vizier.u-strasbg.fr/viz-bin/sed?-c=186.2655972083333%2012.886983138888887&-c.rs=5,414.0
2MASS J13000535+1632148,NVSS J130005+163212,195.02228295163,16.53743305218,Sy1,0.07993,z,AGN|G |IR |Rad|Sy1|X,http://vizier.u-strasbg.fr/viz-bin/sed?-c=195.02228295163%2016.53743305218&-c.rs=5,428.0
NGC 262,NGC 262,12.196426138730002,31.956956641230004,Sy2,0.015260233939595302,v,* |AGN|Bla|G |GiG|IR |QSO|Rad|Sy2|X |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=12.196426138730002%2031.956956641230004&-c.rs=5,484.0
IRAS 04385-0828,IRAS 04385-0828,70.2290037068,-8.37281297812,Sy2,0.015064,z,* |AGN|G |GiG|IR |Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=70.2290037068%20-8.37281297812&-c.rs=5,541.0
NGC 4945,NGC 4945,196.3636625,-49.4679,Sy2,0.0017968563040435814,v,AGN|G |GiG|IR |Rad|Sy2|X |cm |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=196.3636625%20-49.4679&-c.rs=5,628.0
IC 4553,IC 4553,233.73843316666665,23.50322455555555,SyG,0.0184,z,AGN|G |GiG|GiP|IG |IR |LIN|Rad|SyG|X |gam|smm,http://vizier.u-strasbg.fr/viz-bin/sed?-c=233.73843316666665%2023.50322455555555&-c.rs=5,654.0
NGC 5252,NGC 5252,204.56612821846997,4.542644796,Sy2,0.02294,z,AG?|AGN|EmG|G |GiG|Sy1|Sy2|X |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=204.56612821846997%204.542644796&-c.rs=5,773.0
Mrk 279,Mrk 279,208.26431206837998,69.30816969735,Sy1,0.030601,z,AGN|G |IR |QSO|Sy1|UV |X |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=208.26431206837998%2069.30816969735&-c.rs=5,852.0


In [3]:
i = 139
Sample[i]

Main_Id,Id_used_NED,RA,DEC,otype_txt,redshift,rvz_type,otypes,cds_url,SMB_sample_index
object,object,float64,float64,object,float64,str1,object,str93,float64
IC 486,IC 486,120.08743500089999,26.61351567976,Sy2,0.02674,z,AG?|AGN|G |IR |Rad|Sy1|Sy2|X |rG,http://vizier.u-strasbg.fr/viz-bin/sed?-c=120.08743500089999%2026.61351567976&-c.rs=5,16879.0


In [4]:
cds_table = Table.read(path_phot_cds+Sample['Main_Id'][i]+'_CDS_phot_tables'+'.vot', 
                      format='votable')
cds_table

_RAJ2000,_DEJ2000,_tabname,_ID,_tab1_29,_tab1_30,sed_freq,sed_flux,sed_eflux,sed_filter,Bibcode
deg,deg,,,d,s,GHz,Jy,Jy,---,
float64,float64,str32,str64,float64,float64,float64,float32,float32,str32,object
120.087480900,26.613544800,I/289/out,"2UCAC=41188904&-c=120.0874809 +26.6135448,eq=J2000&-c.rs=0.004",--,0.0,138550,0.0142,--,2MASS:Ks,2004AJ....127.3043Z
120.087480900,26.613544800,I/289/out,"2UCAC=41188904&-c=120.0874809 +26.6135448,eq=J2000&-c.rs=0.004",--,0.0,181750,0.0135,--,2MASS:H,2004AJ....127.3043Z
120.087480900,26.613544800,I/289/out,"2UCAC=41188904&-c=120.0874809 +26.6135448,eq=J2000&-c.rs=0.004",--,0.0,241960,0.011,--,2MASS:J,2004AJ....127.3043Z
120.087480800,26.613544700,I/297/out,"NOMAD1===1166-0159320&-c=120.0874808 +26.6135447,eq=J2000&-c.rs=",--,0.0,138550,0.0142,--,2MASS:Ks,2004AAS...205.4815Z
120.087480800,26.613544700,I/297/out,"NOMAD1===1166-0159320&-c=120.0874808 +26.6135447,eq=J2000&-c.rs=",--,0.0,181750,0.0135,--,2MASS:H,2004AAS...205.4815Z
120.087480800,26.613544700,I/297/out,"NOMAD1===1166-0159320&-c=120.0874808 +26.6135447,eq=J2000&-c.rs=",--,0.0,241960,0.011,--,2MASS:J,2004AAS...205.4815Z
120.088492800,26.613057800,I/297/out,"NOMAD1===1166-0159325&-c=120.0884928 +26.6130578,eq=J2000&-c.rs=",--,0.0,138550,0.012,--,2MASS:Ks,2004AAS...205.4815Z
120.088492800,26.613057800,I/297/out,"NOMAD1===1166-0159325&-c=120.0884928 +26.6130578,eq=J2000&-c.rs=",--,0.0,181750,0.0132,--,2MASS:H,2004AAS...205.4815Z


In [6]:
ned_table = Table.read(path_phot_ned+Sample['Main_Id'][i]+'_NED_phot_tables'+'.vot', 
                      format='votable')
ned_table

No.,Observed_Passband,Photometry_Measurement,Uncertainty,Units,Frequency,Flux_Density,Upper_limit_of_uncertainty,Lower_limit_of_uncertainty,Upper_limit_of_Flux_Density,Lower_limit_of_Flux_Density,NED_Uncertainty,NED_Units,Refcode,Significance,Published_frequency,Frequency_Mode,Coordinates_Targeted,Spatial_Mode,Qualifiers,Comments
,,,,,Hz,Jy,,,,,,,,,,,,,,
int32,object,float64,object,object,float64,float64,float64,float64,float64,float64,object,object,object,object,object,object,object,object,object,object
1,14-195 keV (Swift),3.22e-11,+/-0.70E-11,erg/s/cm^2^,2.53e+19,1.27e-07,2.77e-08,2.77e-08,--,--,+/-2.77E-08,Jy,2010ApJS..186..378T,uncertainty,104.50 keV,Broad-band measurement,120.080 +26.638 (J2000),Flux integrated from map,,From new raw data; NED frequency assigned to mid-point ofband in keV
2,14-195 keV (Swift),3.22e-11,,erg/cm^2^/s,2.53e+19,1.27e-07,--,--,--,--,,Jy,2017ApJS..233...17R,no uncertainty reported,104.50 keV,Broad-band measurement,,Modelled datum,Observed flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
3,14-195 keV (Swift),2.99e-11,,erg/cm^2^/s,2.53e+19,1.18e-07,--,--,--,--,,Jy,2017ApJS..233...17R,no uncertainty reported,104.50 keV,Broad-band measurement,,Modelled datum,Intrinsic flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
4,14-195 keV (Swift),3.38e-11,+/-29.55E-12,erg/s/cm^2^,2.53e+19,1.34e-07,1.17e-07,1.17e-07,--,--,+/-1.17E-07,Jy,2018ApJS..235....4O,90% confidence,104.50 keV,Broad-band measurement,120.04926050 +26.62232259 (J2000),From fitting to map,S/N = 13.85 ; power law model,From new raw data; NED frequency assigned to mid-point ofband in keV
5,15-150 keV (Swift),2.8e-11,+/-1.5e-11,erg/cm^2^/s,1.99e+19,1.41e-07,7.54e-08,7.54e-08,--,--,+/-7.54E-08,Jy,2010A&A...524A..64C,uncertainty,82.50 keV,Broad-band measurement,120.066 +26.628 (J2000),Modelled datum,,From new raw data; NED frequency assigned to mid-point ofband in keV
6,14-150 keV (Swift),2.62e-11,,erg/cm^2^/s,1.98e+19,1.32e-07,--,--,--,--,,Jy,2017ApJS..233...17R,no uncertainty reported,82.00 keV,Broad-band measurement,,Modelled datum,Intrinsic flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
7,15-55 keV (Swift),1.79e-11,+/-0.24E-11,erg/cm^2^/s,8.46e+18,2.12e-07,2.84e-08,2.84e-08,--,--,+/-2.84E-08,Jy,2009ApJ...699..603A,uncertainty,35.00 keV,Broad-band measurement,120.099 +26.648 (J2000),Flux integrated from map,,From reprocessed raw data; NED frequency assigned tomid-point of band in keV
8,20-50 keV (Swift),1.02e-11,,erg/cm^2^/s,8.46e+18,1.21e-07,--,--,--,--,,Jy,2017ApJS..233...17R,no uncertainty reported,35.00 keV,Broad-band measurement,,Modelled datum,Intrinsic flux,From reprocessed raw data; NED frequency assigned tomid-point of band in keV


In [22]:
a = np.unique(ned_table['Refcode'])
a

1963CGCG2.C...0000Z
1989H&RHI.C...0000H
1990IRASF.C...0000M
1991RC3.9.C...0000d
1998AJ....115.1693C
2002AJ....124..675C
20032MASX.C.......:
2004SDSS3.C...0000:
2005ApJ...630..759M
2006AJ....131.1163S
2007ApJS..172..599S


In [39]:
t_rem = []
for u_refcode in np.unique(ned_table['Refcode']):
    l_cds = np.where(cds_table['Bibcode'] == u_refcode)[0]#.decode('utf-8'))[0]
    l_ned = np.where(ned_table['Refcode'] == u_refcode)[0]
    if len(l_cds) > 0 and len(l_ned) > 0:
        print('Duplicate!')

Duplicate!
Duplicate!
Duplicate!
Duplicate!


In [42]:
flx_ned = str(ned_table[3]['Flux_Density'])
flx_ned

'1.34e-07'

In [35]:
cds_table[377]

_RAJ2000,_DEJ2000,_tabname,_ID,_tab1_29,_tab1_30,sed_freq,sed_flux,sed_eflux,sed_filter,Bibcode
deg,deg,,,d,s,GHz,Jy,Jy,---,
float64,float64,str32,str64,float64,float64,float64,float32,float32,str32,object
120.087500000,26.613611000,J/MNRAS/456/3335/table12,recno=106,--,0.0,599.59,0.12,--,:=500um,2016MNRAS.456.3335S


In [17]:
for tabindx, tabinfo in enumerate(cds_tablee['_tabname']):
    try:
        time.sleep(0.001)  # This is used to avoid being flag by the server
        name_cat = tabinfo.rpartition('/')[0]
        search = Vizier.query_constraints(catalog='METAcat', name=name_cat)
        cds_tablee['Bibcode'][tabindx] = search[0][0]['bibcode']
    except IndexError:
        print('There is an error at ', tabindx, tabinfo)

In [18]:
cds_tablee

_RAJ2000,_DEJ2000,_tabname,_ID,_tab1_31,_tab1_32,sed_freq,sed_flux,sed_eflux,sed_filter,Bibcode
deg,deg,,,d,s,GHz,Jy,Jy,,
float64,float64,str32,str64,float64,float64,float64,float32,float32,str32,object
150.456380600,28.785966700,I/297/out,"NOMAD1===1187-0182426&-c=150.4563806 +28.7859667,eq=J2000&-c.rs=",--,0.0,138550,0.00501,--,2MASS:Ks,2004AAS...205.4815Z
150.456380600,28.785966700,I/297/out,"NOMAD1===1187-0182426&-c=150.4563806 +28.7859667,eq=J2000&-c.rs=",--,0.0,181750,0.00215,--,2MASS:H,2004AAS...205.4815Z
150.456380600,28.785966700,I/297/out,"NOMAD1===1187-0182426&-c=150.4563806 +28.7859667,eq=J2000&-c.rs=",--,0.0,241960,0.00116,--,2MASS:J,2004AAS...205.4815Z
150.456445000,28.785850000,I/305/out,"GSC2.3===N9AE005113&-c=150.456445 +28.785850,eq=J2000&-c.rs=0.00",--,0.0,382540,0.0021,0.00083,POSS-II:i,2008AJ....136..735L
150.456445000,28.785850000,I/305/out,"GSC2.3===N9AE005113&-c=150.456445 +28.785850,eq=J2000&-c.rs=0.00",--,0.0,468430,0.000992,0.00039,POSS-II:F,2008AJ....136..735L
150.456445000,28.785850000,I/305/out,"GSC2.3===N9AE005113&-c=150.456445 +28.785850,eq=J2000&-c.rs=0.00",--,0.0,640580,0.000753,0.00031,POSS-II:J,2008AJ....136..735L
150.456445000,28.785850000,I/305/out,"GSC2.3===N9AE005113&-c=150.456445 +28.785850,eq=J2000&-c.rs=0.00",--,0.0,674900,0.000364,0.00012,Johnson:B,2008AJ....136..735L
150.456374000,28.785827000,I/317/sample,"PPMXL===4211247675463393029&-c=150.456374 +28.785827,eq=J2000&-c",--,0.0,138550,0.005,0.00015,2MASS:Ks,2010AJ....139.2440R


In [32]:
name_cat = tabinfo.rpartition('/')[0]
name_cat

'J/other/APh/26.282'

In [27]:
search = Vizier.query_constraints(catalog='METAcat', name=name_cat)

In [31]:
search[search.keys()[0]].pprint()

catid kslot catype        name        ...       bibcode       popu authid
----- ----- ------ ------------------ ... ------------------- ---- ------
  255    63      5 J/other/APh/26.282 ... 2006APh....26..282A  255      0


In [29]:
search[0][0]['bibcode']

'2006APh....26..282A'